In [1]:
import pandas as pd
import script_reglas

import nltk as nltk
from emoji import UNICODE_EMOJI
import unicodedata as unicodedata
import re
from __future__ import print_function
import os  # for os.path.basename
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn.manifold import MDS

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
! ls ./datos/


comparacion.csv         etiquetado_pepe.csv     flujos_peticiones.csv
etiquetado.csv          exploracion             results_export_964.xlsx


In [8]:
respuestas=pd.read_excel('./datos/results_export_964.xlsx', encoding='utf-8')

In [9]:
respuestas=respuestas[['Contact UUID', 'URN', 'Name', 'Groups', 'Response 9 (Text) - uncaught']]
respuestas.columns=['id_contacto', 'urn', 'nombre_contacto', 'listas', 'texto']

In [10]:
categoria_reglas=pd.DataFrame()
for i in range(0, respuestas.shape[0]):
    respuesta_regla=pd.DataFrame(script_reglas.procesa_reglas(str(respuestas['texto'].iloc[i])), index=[i])
    categoria_reglas=pd.concat([categoria_reglas, respuesta_regla])
respuestas=pd.concat([respuestas[['id_contacto', 'urn', 'nombre_contacto', 'listas']],
                      categoria_reglas], axis=1)

In [11]:
print(respuestas.shape)
respuestas=respuestas.loc[respuestas.texto!='nan']
print(respuestas.shape)

(3045, 9)
(2472, 9)


In [12]:
respuestas.result.value_counts()

modelo           1012
like-fb           480
si                309
gracias           184
hola              136
no                114
informacion        96
ok                 95
spam               23
twitter-image      13
hasta luego         6
pregunta            4
Name: result, dtype: int64

In [24]:
respuestas.texto.loc[respuestas.result=='pregunta'].value_counts()

???????    1
???        1
??         1
?          1
Name: texto, dtype: int64

In [26]:
respuestas.loc[respuestas.result=='modelo', 'texto']

1       La solicitud de enviar el SMS por Cobrar a 552...
2                                                       🙅
22      Te llame y no pude localizarte. Tramita en lin...
45                                                  59508
47                                                Mi bebe
70      Hola si ya cada integrante tenemos fotos con l...
105     El usuario 2471032511 te envio un mensaje pero...
133     Te llame y no pude localizarte. ¡Ultima oportu...
134     Te llame y no pude localizarte. 3,000 MXN en M...
135                            Mi bebé esta deshidratado 
142                                mensaje fuera de flujo
150                                        En 5m llegamos
157     Tienes saldo congelado, recuperalo recargando ...
160                     ¿Por que mi bebe estornuda mucho?
215                                         Si ya me dijo
227                                               Mi bebe
238                                Ya no estoy embarazada
239           

In [66]:

resultado_reglas=np.array([resultado_reglas[i] for i in nombres])

array(['modelo', 'nan', 'nan', '1', '3',
       'f60ed04c-49af-4d7a-82e2-9a0c7d08fe8e', '+525529070118', 'ALTOPD'],
      dtype='<U36')

In [58]:
resultado_reglas

array({'result': 'modelo', 'texto': 'nan', 'texto_proc': 'nan', 'wc': 1, 'cc': 3, 'id_contacto': 'f60ed04c-49af-4d7a-82e2-9a0c7d08fe8e', 'urn': '+525529070118', 'listas': 'ALTOPD'}, dtype=object)

In [36]:
for i in respuestas:
    resultado_reglas=script_reglas.procesa_reglas(str(i[4]))
    resultado_reglas.update({'id_contacto':i[0],
                             'urn':i[1], 
                            'listas':i[3]})
    

KeyboardInterrupt: 

In [40]:
print(resultado_reglas)

None


In [5]:
label_map={1:'emergencia',
          2:'informacion',
          3: 'nacimiento',
          4: 'respuesta',
          5: 'pregunta',
          6: 'otra',
          7: 'pregunta_busca trabajo',
          8: 'otra_queja'}

In [21]:
#compara[['uncaught_resp_9_text', 'Clasificacion', 'categ_opi']]

In [9]:
pd.crosstab(compara.Clasificacion, compara.categ_opi, margins=True)



categ_opi,emergencia,informacion,nacimiento,otra,otra_queja,pregunta,pregunta_busca trabajo,respuesta,All
Clasificacion,,,,,,,,,
emergencia,5,0,0,0,0,0,0,0,5
informacion,0,37,0,4,0,0,0,4,45
nacimiento,0,0,3,0,0,0,0,0,3
otra,1,29,10,269,126,17,8,78,538
pregunta,4,3,0,4,16,303,53,0,383
All,10,69,13,277,142,320,61,82,974


In [177]:
#compara.uncaught_resp_9_text.loc[(compara.Clasificacion=='pregunta')].ravel()

In [194]:
compara.uncaught_resp_9_text.loc[(compara.Clasificacion=='pregunta') &
                                 (compara.categ_opi==8)].ravel()

array([ 'hola me gustaría saber en dónde puedo poner una queda, ya que se le negó el servicio de atención a mi mamá en dos hospitales',
       '@[173076059407762:274:secretaría de salud] no has respondido a mi pregunta respecto a  porque si hay actualmente una problemática y riesgo de sarampión existe carencia de las vacunas para prevención???',
       'buen día\n\ntengo dudas sobre una denuncia pertinente. \n\nmis vecinos tienen aves de corral sin los debidos requerimientos sobre salubridad, generan cucarachas y otras plagas, además de que el ruido es insoportable. \nquisiera hacer una denuncia pero desconozco dónde o cómo hacer el trámite para que los revise salubridad, ya que de igual forma, utilizan el domicilio como bodega de desechos de verdulería y el aroma es insoportable.',
       'buenas noches, soy de tijuana, me interesa saber en donde puedo reportar a una persona  con tuberculosis, el está conciente de su enfermedad y sin importarle se encuentra todos los días en la salida

In [4]:
respuestas.head()

,id_contacto,urn,nombre,listas,uncaught_resp_9_cat,uncaught_resp_9_val,uncaught_resp_9_text,categ_opi,uncaught_resp_9_text_wc,uncaught_resp_9_text_nchar,uncaught_resp_9_text_mod,es
0,e7e51622-a160-4bb4-9b7d-785c0c2bfbc7,20999,NaN,NaN,Other,La solicitud de enviar el SMS por Cobrar a 552...,la solicitud de enviar el sms por cobrar a 552...,6,24,144,la solicitud de enviar el sms por cobrar a 552...,0
1,313822ff-7541-4db6-8fe1-a252363000ab,522461582793,sarahi,"ALL, ALTOPD",no,no,te llame y no pude localizarte. tramita en lin...,6,24,135,te llame y no pude localizarte tramita en line...,0
2,86962a26-c021-4606-8986-893834ad83bd,524271210385,NaN,"ALL, ALTOPD",Other,59508,59508,4,1,5,59508,0
3,84b7624f-261d-44b4-83ba-6fade1a44c3a,524271654479,NaN,"ALL, ALTOPD",Other,Mi bebe,mi bebe,3,2,7,mi bebe,0
4,e083b2ea-6e2d-410c-b5a4-8067a818d485,522411259330,NaN,"ALL, ALTOPD",Gracias,gracias,hola si ya cada integrante tenemos fotos con l...,3,18,99,hola si ya cada integrante tenemos fotos con l...,0


In [5]:
respuestas.categ_opi.value_counts()

5    458
6    201
8    139
4     84
7     60
1     26
3     13
2     12
Name: categ_opi, dtype: int64

## preprocesamiento

In [6]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text'].str.lower()

 ### Quitar leading y trailing spaces, quitar puntuacion

In [7]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].str.strip()

In [8]:
respuestas['uncaught_resp_9_text_wc'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x).split(" ")))
respuestas['uncaught_resp_9_text_nchar'] = respuestas['uncaught_resp_9_text_mod'].apply(lambda x: len(str(x)))

In [9]:
respuestas['uncaught_resp_9_interrogacion']=respuestas['uncaught_resp_9_text'].str.count('\\?')>0

In [10]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].str.replace('[^\w\s]','')

In [11]:
respuestas['uncaught_resp_9_text_mod']=respuestas['uncaught_resp_9_text_mod'].str.strip()

### Capturar emojis

Actualmente, si el mensaje contiene un emoji, es canalizado a la categoría Emoji. Este filtro debe ser más estricto, pues usualmente ese emoji es usado en un mensaje real

In [12]:
def is_emoji(s):
    count = 0
    for emoji in range(1, len(UNICODE_EMOJI)):
        count += s.count(list(UNICODE_EMOJI.keys())[emoji])
    return count

In [13]:
es_emoji=pd.DataFrame()
for i in range(0, respuestas.shape[0]):
    es=pd.DataFrame({'es':is_emoji(respuestas['uncaught_resp_9_text'].iloc[i])}, index=[0])
    es_emoji=pd.concat([es_emoji, es], ignore_index=True)

In [14]:
respuestas=respuestas.reset_index(drop=True)

In [15]:
respuestas=pd.concat([respuestas, es_emoji], axis=1)

### Quitar emojis que quedan en el texto

In [16]:
def give_emoji_free_text(text):
    allchars = [str for str in text]
    emoji_list = [c for c in allchars if c in UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

In [17]:
for i in range(0, respuestas.shape[0]):
    respuestas['uncaught_resp_9_text_mod'].iloc[i]=give_emoji_free_text(respuestas.uncaught_resp_9_text_mod.iloc[i])

### Quitar  acentos

In [18]:
sinacentos=pd.DataFrame()
for i in range(0, respuestas.shape[0]):
    sinacentos_i=unicodedata.normalize('NFD', respuestas['uncaught_resp_9_text_mod'].iloc[i]).encode('ascii', 'ignore').decode('utf-8')
    sinacentos_i=pd.DataFrame({'texto':sinacentos_i}, index=[i])
    sinacentos=pd.concat([sinacentos, sinacentos_i])

In [19]:
respuestas['uncaught_resp_9_text_mod']=sinacentos

In [20]:
respuestas.uncaught_resp_9_text_mod=respuestas.uncaught_resp_9_text_mod.str.replace('ola|\n|buena noche|buenos dias|buen dia|buenas noches|buenas tardes|buen dia|buena tardes|hola', '')

### Procesar dummy de  listas

In [21]:
respuestas.listas=respuestas.listas.str.lower()
respuestas.listas=respuestas.listas.str.replace(',', '')
respuestas.listas=respuestas.listas.str.replace('con fecha', 'con_fecha')
respuestas.listas=respuestas.listas.str.replace('2 años', '2_anios')

In [23]:
respuestas.listas=respuestas.listas.str.lower()
respuestas.listas=respuestas.listas.str.replace(',', '')
respuestas.listas=respuestas.listas.str.replace('con fecha', 'con_fecha')
respuestas.listas=respuestas.listas.str.replace('2 años', '2_anios')

prueba_listas=respuestas.listas.str.split(expand=True)
lista=[pd.get_dummies(prueba_listas[col]) for col in prueba_listas]

for i in range(0, len(lista)):
    lista[i]['indice'] = lista[i].index
lista=pd.concat(lista, ignore_index=True)
prueba_listas=lista.groupby('indice', as_index=True).agg('sum')

### TF-DF

In [26]:
stop=nltk.corpus.stopwords.words("spanish")

In [27]:
for i in range(0, np.shape(stop)[0]):
    stop[i]=unicodedata.normalize('NFD', stop[i]).encode('ascii', 'ignore').decode('utf-8')

In [ ]:
stemmer = SnowballStemmer("spanish")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='utf-8', ngram_range=([1, 2]),
                        stop_words=stop)
features = tfidf.fit_transform(respuestas.uncaught_resp_9_text_mod).toarray()
labels = respuestas.categ_opi
features.shape

(993, 540)

In [56]:
raros=prueba_listas.sum().loc[prueba_listas.sum()<3].index

In [54]:
#prueba_listas['otros']=prueba_listas[prueba_listas.sum().loc[prueba_listas.sum()<3].index].sum(axis=1)
prueba_listas[raros3].index].sum(axis=1)

indice
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     1.0
16     2.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     1.0
29     0.0
30     0.0
31     0.0
32     0.0
33     0.0
34     0.0
35     1.0
36     0.0
37     1.0
38     0.0
39     0.0
40     0.0
41     0.0
42     0.0
43     0.0
44     1.0
45     0.0
46     0.0
47     0.0
48     0.0
49     0.0
50     0.0
51     0.0
52     0.0
53     0.0
54     0.0
55     0.0
56     0.0
57     0.0
58     0.0
59     0.0
60     0.0
61     0.0
62     0.0
63     0.0
64     0.0
65     0.0
66     0.0
67     0.0
68     0.0
69     0.0
70     0.0
71     0.0
72     0.0
73     0.0
74     0.0
75     0.0
76     0.0
77     0.0
78     0.0
79     0.0
80     0.0
81     0.0
82     0.0
83     0.0
84     0.0
85     0.0
86     0.0
87     0.0
88     0.0
89     0.0
90 

In [33]:
features[1]

array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.6260965 ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  

In [23]:
x_target=respuestas.categ_opi
x_train=respuestas.uncaught_resp_9_text_mod

In [29]:
totalvocab_stemmed = []
totalvocab_tokenized = []
lista_fin=list()
for i in x_train:
    allwords_stemmed = tokenize_and_stem(i) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

(993, 540)

In [64]:
from sklearn.feature_selection import chi2

N = 5
for category_id in sorted(respuestas.categ_opi.unique()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(category_id))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# '1':
  . Most correlated unigrams:
. sangre
. cirugia
. urgente
. normal
. bebe
  . Most correlated bigrams:
. seguro popular
. podrian decir
. hospital general
. puedo hacer
. espero respuesta
# '2':
  . Most correlated unigrams:
. via
. registro
. saben
. pido
. mensajes
  . Most correlated bigrams:
. seguro popular
. gustaria saber
. vacuna hepatitis
. quisiera saber
. secretaria salud
# '3':
  . Most correlated unigrams:
. llama
. hijos
. ami
. nacio
. bebe
  . Most correlated bigrams:
. seguro popular
. gustaria saber
. vacuna hepatitis
. quisiera saber
. secretaria salud
# '4':
  . Most correlated unigrams:
. darme
. voy
. excelente
. mujer
. alta
  . Most correlated bigrams:
. seguro popular
. gustaria saber
. vacuna hepatitis
. quisiera saber
. secretaria salud
# '5':
  . Most correlated unigrams:
. vacunacion
. vacunas
. saber
. vacuna
. hepatitis
  . Most correlated bigrams:
. bolsa trabajo
. seguro popular
. puedo conseguir
. quisiera saber
. vacuna hepatitis
# '6':
  . Mo

In [100]:
from sklearn.decomposition import TruncatedSVD

pca = TruncatedSVD(n_components = 20)
Xall = pca.fit_transform(features)

Xall=pd.DataFrame(Xall)

In [104]:
pd.concat([Xall, respuestas[['uncaught_resp_9_text', 'uncaught_resp_9_text_wc',
                            'listas', 'es', 'uncaught_resp_9_interrogacion', 'categ_opi']]], axis=1).to_csv('./datos/exploracion')

In [65]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(features, x_target)



In [66]:
test=respuestas.uncaught_resp_9_text_mod

In [67]:
tfidf_matrix_test = tfidf.transform(test)



In [82]:
print(clf.predict(tfidf.transform(['mi bebe ya nacio se llama sebastiana'])))

[5]


In [68]:
pd.DataFrame(clf.predict(features))[0].value_counts()



5    823
6     85
8     71
7     14
Name: 0, dtype: int64

In [69]:
test.loc[clf.predict(features)==7]

109     soy de cd juarezsoy nutriologa y megustaria a...
113                                     bolsa de trabajo
130    una pregunta para ver la bolsa de trabajo que ...
185      quisiera saber si hay alguna bolsa de trabaj...
193     que tal estara abierta la bolsa de trabajo en...
204      me podria decir si esta habierta la bolsa de...
276     tienen vacantes de enfermeria para mexicali o...
398    buena tarde licenciad es un gusto poder saluda...
440        tienen abierta bolsa de trabajo soy enfermero
451     para informacion de bolsa de trabajo para enf...
517      mi nombre es leticia martinez hernandez soy ...
639     esta abierta la bolsa de trabajo para enfermeria
715    secretaria de gobierno de cd de mexicopresente...
930             hay bolsa de trabajo abierta en cdjuarez
Name: uncaught_resp_9_text_mod, dtype: object

In [310]:
terms = tfidf_vectorizer.get_feature_names()